In [ ]:
import mysql.connector

try:
    db = mysql.connector.connect(
        host="localhost",
        user="root",
        password="aio2025",
        database="aio2025"
    )
    print("Connection successful!")

except mysql.connector.Error as err:
    print(f"Error: {err}")
    exit()

In [ ]:
cursor = db.cursor()
sql_task = """
"""

try:
    cursor.execute("DROP TABLE IF EXISTS tasks;")
    db.commit()

    cursor.execute(sql_task)
    db.commit()
except mysql.connector.Error as err:
    print(err)

    